In [3]:
import os
from email.parser import Parser

def emails_between(suspects):
    emails_between = []
    email_directory = "enron_email_corpus"
    
    parser = Parser()

    # Process suspects for efficient lookup
    all_suspects = {email.lower(): account for account, emails in suspects.items() for email in emails}

    for root, dirs, files in os.walk(email_directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                    msg = parser.parse(file)

                sender = msg.get('From')
                recipients = msg.get('To')

                if sender and recipients:
                    # Process recipients to ensure lowercase and strip spaces
                    processed_recipients = [email.strip().lower() for email in recipients.split(',') if email.strip()]

                    # Check if the sender and recipient is a suspect
                    sender_match = sender.lower() in all_suspects
                    recipient_matches = False
                    unique_recipients = []
                    for recipient in processed_recipients:
                        if recipient in all_suspects:
                            recipient_matches = True
                        if recipient not in unique_recipients:
                            unique_recipients.append(recipient)

                   # Check if any recipients that are suspects are limited to 20 or fewer unique recipients
                    if sender_match and recipient_matches and len(unique_recipients) <= 20:
                        emails_between.append(msg)
            except Exception as e:
                print(f"Failed to parse {file_path}: {e}")
                continue

    return emails_between

